# Week 4 - Clustering Universities based on neighborhood similarity

#### Objective: In this notebook, I will analyze universities' neighborhoods (from USA) using foursquare data. Therefore, we can classify universities based on its surrounds. This could help young people select a university with a neighborhood with the qualities that they prefer, and the universities by letting they know possible attractiveness points. With this information both future college students and university administrators could improve their decision making. And their a third public to this analysis: entrepreneurs seeking to invest in a business related with college audience. They could see the classes od university and explore possible demands.
#### Data: This could be done by one data frame with a list of universities and its latitude and longitude. This [website](https://www.latlong.net/category/universities-236-47.html) contains exactly what we need. Then, with foursquare API we can make a request to access data of local venues close to each university. 
#### Methodology: With all data merged, we can use K-Nearest Neighborhood analyzes to identify the similar classes of universities based on their venues commercial/entertaining places.  We also plot a map to visualize their distribution in space (final part of the notebook).
#### Results and discussion: based on the results of the KNN algorithm, we can see that most universities fall on the same class: with a lot of quick meals shops (pizza, coffee, bagel) and college related entertainment. I.e., 77,5% of the colleges have similar neighborhoods. This number is considerable high, means that universities could favorable certain business. Therefore, the other categories might need business or could be a unique factor to differentiate the university from others.  
#### Conclusion: The fact that most university neighborhoods are similar may mean that profiles that do not fit may present a business opportunity or be a differential factor for certain student profiles.
## Below we describe the data and steps taken:


## Step 0: Load all libraries

In [74]:
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Step 1: get a data frame of some universities

This is possible through the url: https://www.latlong.net/category/universities-236-47.html

In [81]:
# Web scraping wiki page
url = "https://www.latlong.net/category/universities-236-47.html"

website_url = requests.get(url).text

soup = BeautifulSoup(website_url,"lxml")

table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

df = df[0]

df.columns=['university','latitude','longitude']

df.head()

,university,latitude,longitude
0,"Utah Valley University, Orem, UT, USA",40.277779,-111.713890
1,"Indiana University of Pennsylvania, Indiana, P...",40.617001,-79.160004
2,"University of Illinois, Champign - Urbana, Ili...",40.110558,-88.228333
3,"Massachusetts Institute of Technology, Cambrid...",42.360001,-71.092003
4,"Stanford University, CA, USA",37.428230,-122.168861


## Step 2: get foursquare data

First, we need to know the coords to centralize folium map

In [76]:
address = 'USA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of USA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of USA are 39.7837304, -100.4458825.


Foursquare client:

In [11]:
CLIENT_ID = 'XXX'your Foursquare ID
CLIENT_SECRET = 'XXX'#your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Creating a function to call all venues

In [83]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
        

In [84]:
uni_venues = getNearbyVenues(df['university'], df['latitude'], df['longitude'])

Utah Valley University, Orem, UT, USA
Indiana University of Pennsylvania, Indiana, PA, the USA
University of Illinois, Champign - Urbana, Ilinois, the US
Massachusetts Institute of Technology, Cambridge, MA, USA
Stanford University, CA, USA
Harvard University, Cambridge, MA, USA
California Institute of Technology, Pasadena, CA, USA
University of Chicago, Chicago, IL, USA
Eastern University, St Davids, PA, USA
University of Michigan, Ann Arbor, MI, USA
The University of Massachusetts, Amherst, MA, USA
Penn State University, State College, PA, USA
Lawrence University, Appleton, WI, USA
Quinnipiac, QU School of Law, Hamden, CT, US
Towsley Center for Continuing Medical Education, Ann Arbor, MI, US
Villanova University, Villanova, PA, USA
Boise State University, Boise, ID, US
University of Washington, Seattle, WA, USA
University of North Carolina at Chapel Hill, Chapel Hill, NC, USA
Madonna University, Livonia, MI, USA
University of California, Santa Barbara, CA, USA
University of Californi

In [85]:
uni_venues.shape

(1114, 7)

### Preparing the data frame to cluster

In [86]:
# one hot encoding
uni_onehot = pd.get_dummies(uni_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
uni_onehot['Neighborhood'] = uni_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [uni_onehot.columns[-1]] + list(uni_onehot.columns[:-1])
uni_onehot = uni_onehot[fixed_columns]

uni_onehot.head()

,Neighborhood,ATM,Accessories Store,American Restaurant,Amphitheater,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Garden,Beer Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Baseball Diamond,College Basketball Court,College Bookstore,College Cafeteria,College Engineering Building,College Football Field,College Gym,College Library,College Quad,College Rec Center,College Residence Hall,College Stadium,College Theater,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Electronics Store,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Lingerie Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Mosque,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Observatory,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Resort,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Squash Court,Stadium,State / Provincial Park,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,University,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Utah Valley University, Orem, UT, USA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Utah Valley University, Orem, UT, USA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Utah Valley University, Orem, UT, USA",0,0,0,0,0,0,0,0,0,0,0,0,0,

In [88]:
uni_grouped = uni_onehot.groupby('Neighborhood').mean().reset_index()

uni_grouped.rename(columns={'Neighborhood':'university'},inplace=True)

uni_grouped.shape

(47, 212)

In [93]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['university']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['university'] = uni_grouped['university']

for ind in np.arange(uni_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(uni_grouped.iloc[ind, :], num_top_venues)
    
    
neighborhoods_venues_sorted.shape

(47, 11)

In [89]:
# set number of clusters
kclusters = 5

uni_grouped_clustering = uni_grouped.drop('university', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(uni_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 0, 2, 0, 2, 2, 2])

In [90]:
df['university'].nunique()-uni_grouped['university'].nunique()

2

In [92]:
df0=df[df.university.isin(uni_grouped['university'])]
df0.shape

(47, 3)

In [94]:
uni_merged = df0

# add clustering labels
uni_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
uni_merged = uni_merged.join(neighborhoods_venues_sorted.set_index('university'), on='university')

uni_merged.head() # check the last columns!

<ipython-input-94-22260a1c5728>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_merged['Cluster Labels'] = kmeans.labels_


,university,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Utah Valley University, Orem, UT, USA",40.277779,-111.713890,2,Fast Food Restaurant,Hotel,Juice Bar,College Library,College Gym,Bus Station,Baseball Stadium,Theater,Breakfast Spot,Video Store
1,"Indiana University of Pennsylvania, Indiana, P...",40.617001,-79.160004,2,Fast Food Restaurant,Coffee Shop,Food Service,American Restaurant,Video Store,Sandwich Place,Pizza Place,Mediterranean Restaurant,Farmers Market,Falafel Restaurant
2,"University of Illinois, Champign - Urbana, Ili...",40.110558,-88.228333,2,Coffee Shop,Chinese Restaurant,Sandwich Place,Mexican Restaurant,Pizza Place,Korean Restaurant,Bagel Shop,Clothing Store,Fast Food Restaurant,Café
3,"Massachusetts Institute of Technology, Cambrid...",42.360001,-71.092003,2,Coffee Shop,Pizza Place,Park,College Gym,Burrito Place,Greek Restaurant,Soccer Field,Concert Hall,Food Truck,New American Restaurant
4,"Stanford University, CA, USA",37.428230,-122.168861,0,Coffee Shop,Fountain,Sculpture Garden,Café,Monument / Landmark,Sandwich Place,Food Court,Mexican Restaurant,Plaza,Chinese Restaurant


## Vizualize clusters in a map

In [96]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(uni_merged['latitude'], uni_merged['longitude'], uni_merged['university'], uni_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyzing each group
### Group 0

In [97]:
uni_merged.loc[uni_merged['Cluster Labels'] == 0, uni_merged.columns[[1] + list(range(5, uni_merged.shape[1]))]]

,latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,37.428230,Fountain,Sculpture Garden,Café,Monument / Landmark,Sandwich Place,Food Court,Mexican Restaurant,Plaza,Chinese Restaurant
6,34.138000,Beer Garden,Coffee Shop,Restaurant,Fountain,Lake,Theater,Scenic Lookout,Park,Yoga Studio
18,35.904613,Coffee Shop,College Baseball Diamond,Bus Station,Volleyball Court,Monument / Landmark,Fast Food Restaurant,Cafeteria,Sandwich Place,Social Club
27,33.579166,College Basketball Court,Park,College Rec Center,Lawyer,Food Service,Sandwich Place,Optical Shop,College Cafeteria,Yoga Studio
35,32.593357,Farmers Market,Coffee Shop,Food Court,Flower Shop,Fast Food Restaurant,Falafel Restaurant,Empanada Restaurant,Electronics Store,Donut Shop
44,37.301304,Café,Fast Food Restaurant,Coffee Shop,Sandwich Place,Mexican Restaurant,Boutique,Women's Store,Furniture / Home Store,Japanese Restaurant


## Group 1


In [98]:
uni_merged.loc[uni_merged['Cluster Labels'] == 1, uni_merged.columns[[1] + list(range(5, uni_merged.shape[1]))]]

,latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,42.283833,Café,College Residence Hall,Bagel Shop,Pool,Sandwich Place,Observatory,Food Court,Intersection,Gymnastics Gym


## Group 2


In [104]:
uni_merged.loc[uni_merged['Cluster Labels'] == 2, uni_merged.columns[[1] + list(range(5, uni_merged.shape[1]))]]

,latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,40.277779,Hotel,Juice Bar,College Library,College Gym,Bus Station,Baseball Stadium,Theater,Breakfast Spot,Video Store
1,40.617001,Coffee Shop,Food Service,American Restaurant,Video Store,Sandwich Place,Pizza Place,Mediterranean Restaurant,Farmers Market,Falafel Restaurant
2,40.110558,Chinese Restaurant,Sandwich Place,Mexican Restaurant,Pizza Place,Korean Restaurant,Bagel Shop,Clothing Store,Fast Food Restaurant,Café
3,42.360001,Pizza Place,Park,College Gym,Burrito Place,Greek Restaurant,Soccer Field,Concert Hall,Food Truck,New American Restaurant
5,42.374443,Pizza Place,Gastropub,Bookstore,Coffee Shop,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Burger Joint,Ice Cream Shop
7,41.789722,Coffee Shop,Bookstore,Sandwich Place,History Museum,Food Truck,College Cafeteria,Student Center,Mexican Restaurant,Frozen Yogurt Shop
8,40.051529,College Quad,Café,Dance Studio,Yoga Studio,Discount Store,Flower Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
9,42.278046,Bar,Sandwich Place,Pizza Place,Noodle House,American Restaurant,Clothing Store,Bakery,Ice Cream Shop,Burger Joint
10,42.391155,Bus Stop,Coffee Shop,College Auditorium,College Cafeteria,Café,Snack Place,College Arts Building,Pub,Sandwich Place
11,40.798214,Sandwich Place,Clothing Store,Pizza Place,American Restaurant,Coffee Shop,Ice Cream Shop,Café,Salon / Barbershop,Burger Joint


## Group 3


In [100]:
uni_merged.loc[uni_merged['Cluster Labels'] == 3, uni_merged.columns[[1] + list(range(5, uni_merged.shape[1]))]]

,latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,38.033554,College Gym,Fast Food Restaurant,Café,Clothing Store,Electronics Store,Coffee Shop,Burger Joint,Historic Site,Hotel


## Group 4


In [101]:
uni_merged.loc[uni_merged['Cluster Labels'] == 4, uni_merged.columns[[1] + list(range(5, uni_merged.shape[1]))]]

,latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,44.260445,American Restaurant,ATM,Music Store,History Museum,Sandwich Place,Chinese Restaurant,Thai Restaurant,Fast Food Restaurant,Farmers Market
